In [2]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("chinook.db")


query = """
SELECT c.CustomerId, c.FirstName, c.LastName, COUNT(i.InvoiceId) as InvoiceCount
FROM customers c
JOIN invoices i ON c.CustomerId = i.CustomerId
GROUP BY c.CustomerId
"""
df_invoice_counts = pd.read_sql(query, conn)

print("Invoice Counts per Customer:")
print(df_invoice_counts.head())

conn.close()

DatabaseError: Execution failed on sql '
SELECT c.CustomerId, c.FirstName, c.LastName, COUNT(i.InvoiceId) as InvoiceCount
FROM customers c
JOIN invoices i ON c.CustomerId = i.CustomerId
GROUP BY c.CustomerId
': no such table: customers

In [4]:

df_movie = pd.read_csv("movie.csv")


df_color = df_movie[['director_name', 'color']].drop_duplicates()
df_reviews = df_movie[['director_name', 'num_critic_for_reviews']].drop_duplicates()


df_left = pd.merge(df_color, df_reviews, on='director_name', how='left')

df_outer = pd.merge(df_color, df_reviews, on='director_name', how='outer')

print("\nLeft Join Row Count:", len(df_left))
print("Outer Join Row Count:", len(df_outer))


Left Join Row Count: 5547
Outer Join Row Count: 5547


In [5]:
df_titanic = pd.read_excel("titanic.xlsx")
df_class_stats = df_titanic.groupby('Pclass').agg({
    'Age': 'mean',
    'Fare': 'sum',
    'PassengerId': 'count'
}).rename(columns={'PassengerId': 'Count'})

print("\nTitanic Class Statistics:")
print(df_class_stats)


Titanic Class Statistics:
              Age        Fare  Count
Pclass                              
1       38.233441  18177.4125    216
2       29.877630   3801.8417    184
3       25.140620   6714.6951    491


In [ ]:
df_movie_group = df_movie.groupby(['color', 'director_name']).agg({
    'num_critic_for_reviews': 'sum',
    'duration': 'mean'
})

print("\nMovie Group Stats:")
print(df_movie_group.head())

In [ ]:
df_flights = pd.read_parquet("flights.parquet")

df_flight_stats = df_flights.groupby(['Year', 'Month']).agg({
    'FlightNum': 'count',
    'ArrDelay': 'mean',
    'DepDelay': 'max'
})

print("\nFlight Statistics by Year/Month:")
print(df_flight_stats.head())

In [ ]:
def age_group(age):
    return 'Child' if age < 18 else 'Adult'

df_titanic['Age_Group'] = df_titanic['Age'].apply(age_group)
print("\nAge Groups Added:")
print(df_titanic[['Age', 'Age_Group']].head())

In [ ]:
df_employee = pd.read_csv("employee.csv")

def normalize(group):
    group['NormalizedSalary'] = group['Salary'] / group['Salary'].max()
    return group

df_normalized = df_employee.groupby('Department').apply(normalize)
print("\nNormalized Salaries:")
print(df_normalized.head())

In [ ]:
def duration_class(duration):
    if duration < 60: return 'Short'
    elif 60 <= duration <= 120: return 'Medium'
    else: return 'Long'

df_movie['Duration_Class'] = df_movie['duration'].apply(duration_class)
print("\nMovie Duration Classes:")
print(df_movie[['duration', 'Duration_Class']].head())

In [ ]:
def titanic_pipeline(df):
    return (df
        .query('Survived == 1')
        .assign(Age=lambda x: x['Age'].fillna(x['Age'].mean()),
                Fare_Per_Age=lambda x: x['Fare'] / x['Age'])
    )

df_titanic_clean = titanic_pipeline(df_titanic)
print("\nTitanic Pipeline Result:")
print(df_titanic_clean.head())

In [ ]:
def flights_pipeline(df):
    return (df
        .query('DepDelay > 30')
        .assign(Delay_Per_Hour=lambda x: x['DepDelay'] / (x['AirTime']/60))
    )

df_flights_clean = flights_pipeline(df_flights)
print("\nFlights Pipeline Result:")
print(df_flights_clean.head())